In [25]:
# Implements the Doubly robust value estimator for the learned policies

In [26]:
import pandas as pd
import numpy as np
import pickle

In [27]:
df_train = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [28]:
df_val = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [29]:
df_test = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [30]:
# load in the policies for the physician on val and test sets
phys_policy_val = pickle.load(open("val_policy.p", "rb" ))
phys_policy_test = pickle.load(open("test_policy.p", "rb" ))

In [31]:
# get the indices for indexing into the policy matrix. 
indices = phys_policy_test[:,4].astype(int) + 5
phys_action_probs = phys_policy_test[range(len(phys_policy_test)),indices]

In [32]:
# Two cells below - load the actions and q values associated with the agent policy - these are outputted by the
# neural network used to learn the optimal policy and eval_policy respectively.

In [33]:
agent_actions = pickle.load(open('../continuous/dqn_normal/dqn_normal_actions_test.p', "rb" ))

In [34]:
agent_qsa = pickle.load(open('../continuous/dqn_normal/dqn_normal_Qsa_test.p', "rb" ))
agent_qsa = np.array(agent_qsa)
phys_qsa = pickle.load(open('../continuous/phys_sarsa/phys_qsa_test.p', "rb" ))
phys_qsa = np.array(phys_qsa)

In [35]:
agent_q = pickle.load(open('../continuous/dqn_normal/dqn_normal_q_test.p', "rb" ))

In [36]:
agent_reward_estimates = pickle.load(open('./test_rewards.p', "rb" ))


In [37]:
df_test['agent_action'] = np.array(agent_actions)

In [38]:
df_test['agent_q'] = np.array(agent_q)

In [39]:
df_test['phys_prob'] = phys_action_probs

In [40]:
df_test['reward_estimate'] = agent_reward_estimates  #R_had（？，1）

In [41]:
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [42]:
df_test.shape

(55506, 66)

In [43]:
unique_ids = df_test['icustayid'].unique()

In [71]:
def val_estimate_recurse(traj,t):
    #在论文（Improving Sepsis Treatment Strategies by Combining Deep and Kernel-Based Reinforcement Learning）中
    # V_hat、Q_hat 可以选用行为策略和DQN策略中的值
    
    ind = traj.index[t-1]
    V_hat =agent_qsa[ind].max() #df_test.loc[ind,'reward_estimate'] 

    iv = df_test.loc[ind,'iv_input']
    vaso = df_test.loc[ind, 'vaso_input']
    phys_action = action_map[(iv,vaso)]           # pb action
    phys_prob = df_test.loc[ind,'phys_prob']

    agent_action = traj.loc[ind,'agent_action']
    #q = agent_qsa[ind,agent_action]
    Q_hat = agent_qsa[ind][phys_action] 
    #Q_hat = traj.loc[ind, 'agent_q']
    reward = traj.loc[ind, 'reward']
    
    # calculate rho - the importance sampling factor    
#     TODO Consider making this stochastic?
    if agent_action == phys_action:
        rho = 1.0/phys_prob
    else:
        rho = 0
        
    if t == len(traj):
        val_estimate = 0
    else:
        val_estimate = val_estimate_recurse(traj, t+1)
    
    return V_hat + rho*(reward + gamma*val_estimate - Q_hat)
#                V_hat(s)   + rho (r_t    +   gamma *val_estimate - Q_hat(s,a) )
#              1. V_hat(s) = Sigma_a{Pe(a\s)*Q_hat(s,a)}
#              2. 通过 max_a（Q_hat(s,a)）得到
#              3. NN近似得到，先学习预测模型通过预测模型预测V_hat(s)

In [72]:
values = [] # contains the value estimates for each trajectory
# This discount factor (gamma) should be the same as that used to find Q(s,a) for the physician policy (sarsa_physician)
# and in eval_policy
gamma = 0.99
for uid in unique_ids:
    trajectory = df_test.loc[df_test['icustayid'] == uid]
    cur_val = val_estimate_recurse(trajectory, 1)
    if cur_val > 15 or cur_val < -15:
        continue
    values.append(cur_val)

8.813811 9.39233
13.41852 14.473984
9.479704 10.485052
9.36529 10.041044
10.172912 10.4292965
9.411376 10.132199
9.094264 9.094264
3.8364081 15.535258
6.595299 14.830901
11.002248 14.080907
9.98815 13.345394
7.0669994 10.040085
9.025439 12.323787
9.771938 9.771938
9.991545 12.966583
11.621918 13.408753
12.008785 12.008785
14.415731 14.415731
11.96037 11.96037
8.548071 10.807883
9.366753 12.598269
10.052422 10.849054
10.314625 10.878498
13.7432575 15.092049
14.620731 15.197901
9.831158 15.27982
9.529488 14.986546
11.984117 11.984117
5.6656156 10.488148
4.8933945 9.657877
1.875973 12.438737
5.3899326 11.778492
5.053315 11.717901
4.2094336 9.944537
4.724646 9.8372135
7.366295 9.24227
7.6525264 9.175296
7.5013676 8.931954
7.688989 8.811392
13.6879635 19.957085
13.4583645 16.966244
13.337286 14.332389
10.699684 10.699684
11.147463 11.147463
8.194744 13.365637
11.109499 15.523609
5.212314 16.05912
16.368479 16.368479
14.354165 18.212934
14.578447 19.33004
11.180847 16.513643
9.492096 15.8840

10.006214 10.006214
9.703342 10.507629
9.172317 9.752514
8.851564 10.420304
11.198897 11.478392
10.983988 11.740769
10.961281 12.098568
11.529692 12.349357
9.253929 12.872032
9.359397 13.288434
10.602064 13.311836
9.844835 13.374271
8.30625 10.1838
11.108648 14.526433
10.6421 15.106147
12.184044 14.80999
11.814192 14.817759
10.530928 11.921299
9.129343 14.703207
8.237436 15.227305
7.8560133 14.1230135
7.735239 12.674533
10.566822 12.57039
7.0986724 12.038305
8.770355 14.870932
9.361563 10.321478
7.2065773 10.7191725
6.781206 12.059889
10.860819 12.092049
8.063431 12.791816
10.38472 10.38472
7.9943275 10.802551
8.777666 8.845655
9.334305 10.012729
11.35201 11.35201
11.7602005 11.7602005
10.840874 17.844635
11.887754 15.164896
7.591481 10.4046755
9.828891 15.116617
8.66152 10.073978
9.167614 14.745946
15.377157 15.377157
8.317076 15.759466
8.281371 15.121754
10.728585 14.323834
8.488576 14.85603
9.076593 14.799609
5.5442963 15.013896
8.93579 10.975468
10.011988 13.411968
8.441275 10.5655

9.216607 10.807223
10.205407 10.205407
11.608316 11.608316
12.419205 12.419205
10.157012 10.157012
8.763516 12.921472
3.7258706 11.272766
12.363214 13.543719
9.748375 11.446548
8.763405 13.676157
12.000372 13.536127
11.804096 12.886063
10.230507 13.363186
11.633543 12.438287
9.836098 12.904173
8.346521 15.276968
8.30044 13.972052
10.647766 11.409506
11.214946 13.93779
11.04939 13.826636
9.560987 17.59787
5.406704 14.116871
9.657976 15.797382
9.894217 13.865516
9.374471 12.745447
9.435706 12.238492
9.743824 13.452282
9.707617 12.316708
10.637184 12.75583
9.471775 15.047609
10.693474 13.340869
10.505173 13.714804
10.917909 14.230009
11.595504 15.106245
13.749101 15.241843
11.924792 15.813405
7.419853 10.740701
8.379404 11.599371
9.1886425 13.318796
10.637638 13.697607
16.202541 18.972805
10.011745 12.169296
12.087084 13.385195
13.060572 13.850979
14.222567 14.222567
6.179064 11.794201
7.71806 12.447746
6.2533712 12.005451
10.302392 10.71729
6.469887 10.970574
8.112698 10.575338
7.168562 

9.028491 12.368063
10.082469 12.5750065
9.686864 12.926382
11.769347 15.910902
10.426026 13.075572
9.479288 13.06964
14.357487 14.357487
9.779287 11.212662
8.934349 9.684385
6.660078 9.418978
5.491788 7.8094096
5.775836 8.552456
7.9090586 11.43448
4.7722588 10.242925
5.320733 9.560152
8.823198 11.784337
7.2372084 12.111423
8.225906 12.516895
8.02726 12.580226
7.377364 12.154028
7.5078206 10.473272
6.7097692 10.600789
8.166403 10.948926
8.04217 11.243761
8.981743 11.677858
7.8281584 11.521199
10.865886 13.073299
10.608797 11.702433
10.168554 10.168554
10.119701 10.119701
8.952543 9.06786
9.66638 9.66638
9.786792 9.786792
8.803651 9.958767
8.234018 8.834854
8.1518755 8.344437
9.665043 10.940036
8.461563 10.189828
8.205059 9.825247
6.7822876 9.347976
7.0309453 9.068969
7.328828 9.06629
7.281906 8.34618
9.73451 9.73451
6.381357 7.8849635
5.98459 6.9080677
10.580545 15.467407
7.1508474 14.0120125
8.032747 15.476468
4.9537206 10.947045
8.092399 11.4767685
1.5530934 11.384475
4.5038233 11.729

12.657649 12.729305
8.688448 12.121846
6.9225698 9.890789
4.5740314 8.581362
4.391367 8.017277
7.876729 7.876729
7.685571 7.685571
7.3926954 7.966813
2.9624834 7.491972
6.265784 8.580451
7.5988026 9.234814
8.118165 9.358681
8.780573 9.311933
8.368822 12.100185
9.138264 13.788106
6.036581 8.125188
7.517828 9.375841
7.5292354 9.033361
6.470167 8.600011
5.3442574 9.269812
9.197372 9.197372
7.495489 13.134119
7.525516 12.2734375
13.026041 13.026041
9.761487 16.948507
11.913403 11.913403
9.0786495 12.860758
10.244558 12.437401
10.274759 13.225752
8.585836 11.379816
13.073286 13.073286
11.785826 11.785826
13.295668 13.295668
12.641942 14.804112
7.2802324 10.457319
6.4331064 7.7215023
8.392293 10.790554
5.5329094 8.206512
12.421027 12.421027
11.727428 11.727428
9.163565 10.264495
12.39753 12.39753
6.953057 10.959871
8.642909 14.125966
6.9252944 10.7026005
9.537325 12.159676
8.803734 14.398418
11.884325 11.884325
8.949039 13.754892
15.296083 15.296083
10.298534 13.33871
-4.571727 11.571089
3.0

...

10.410645 10.410645
11.925816 11.991533
12.701498 13.708419
11.993456 13.097141
12.421467 12.421467
7.7733507 13.490923
5.680402 14.055631
8.519308 14.127163
7.8319054 13.989117
7.4206877 13.853069
2.2179735 14.230986
2.298293 14.248935
3.6815648 12.931631
-0.9784341 12.26445
1.3728957 12.065563
1.3621438 11.968943
10.5624695 10.718477
10.363024 10.712254
9.499096 12.189613
10.834092 10.834092
7.8978662 11.26872
5.4910564 9.352392
5.855462 9.036192
6.2298765 10.221067
9.358199 10.261592
13.202248 13.202248
13.413562 13.413562
9.320666 16.178688
9.25239 10.183567
7.459843 12.774442
5.8648076 9.059761
6.201776 7.817975
6.216096 7.830941
9.265366 9.265366
10.805159 16.213768
10.013941 15.985879
9.922264 15.661984
9.803307 15.840509
9.762871 15.881483
9.779617 16.184223
6.825646 13.099344
6.9856515 13.063089
7.1216936 12.981327
7.3888845 12.9814415
7.9689264 14.294903
8.945872 14.10854
12.500006 12.500006
12.6300125 13.614143
8.080682 13.472747
5.589307 9.503398
4.3905134 10.566462
7.18446

9.712888 13.374514
10.522415 12.041662
9.93576 11.811979
9.366753 11.265916
8.9075575 10.355024
10.206734 12.214139
10.213637 10.562662
9.00292 10.441149
8.812771 10.690857
9.741555 12.594803
9.709653 10.781952
10.195517 11.382169
8.733259 11.6629925
8.895426 11.558531
8.725525 11.445421
9.063198 12.086558
9.137281 12.03654
10.138765 14.161612
9.772798 9.964736
7.235155 11.942563
10.524641 10.524641
9.047115 10.640217
11.035313 11.035313
10.15782 10.15782
11.143103 11.143103
7.2589474 9.734125
9.32402 10.437916
11.586248 11.586248
13.895589 14.652147
8.822708 12.202954
12.117953 14.738986
12.975575 13.687983
12.664892 12.664892
9.248741 11.496849
9.900581 10.509244
9.404005 11.683187
9.556393 13.860724
7.462275 11.58738
9.620785 11.408482
9.91254 11.814411
11.187518 11.187518
11.300051 11.300051
12.265843 12.265843
11.548801 12.419268
12.711483 13.88833
13.009759 14.739265
15.5346775 16.30858
8.194585 12.348021
0.10204077 10.720301
8.8346615 9.57457
8.239003 10.928413
8.496062 10.98228

8.357637 13.024508
6.148283 9.4437475
10.885575 15.487202
9.332533 14.7077675
11.746233 14.203325
11.061323 14.941473
9.154247 13.614063
9.831978 11.817524
9.9105625 12.431122
10.763964 12.895429
10.782955 11.914698
11.210205 13.402145
10.320175 13.4776325
10.956375 13.079186
11.318985 13.590982
7.9705534 16.82828
5.4322953 14.368784
4.737626 15.045708
11.565647 14.265854
10.1910515 11.422597
12.211011 13.931052
12.979773 16.29131
12.556538 13.974924
10.808937 10.808937
10.205263 10.205263
9.766838 12.755226
9.702974 10.526602
8.866341 8.866341
10.240854 10.240854
8.793612 10.674797
11.041445 11.041445
11.724597 11.724597
11.250198 11.250198
7.2358766 12.536952
3.267939 16.72282
8.808 13.874914
4.374779 9.686958
-0.4928046 8.190558
7.8443146 8.95434
2.1950097 8.293664
8.128937 12.397236
2.7599492 9.3210335
0.42534912 8.451284
3.9506302 9.651278
-0.08810949 11.495535
-3.4701233 11.715915
13.001229 13.001229
7.6458426 11.472363
8.2155695 11.72884
7.6186333 11.341601
8.234119 11.602559
8.

3.8071277 9.288722
5.6067224 10.040929
6.0419283 10.749628
7.1514225 9.880524
7.3495574 9.94595
6.9105277 16.738295
3.4381008 9.929307
2.1464963 7.64684
2.1319818 6.9247894
3.77141 7.841297
3.3976417 7.1934385
5.772395 8.836464
7.408631 8.661919
3.63108 7.6786375
4.815237 8.572713
4.5446067 7.770639
7.211544 8.001731
9.609014 12.476442
6.9121184 7.0443907
6.2679086 8.107428
6.3744316 8.563995
7.0061226 8.733107
6.9600816 9.994935
6.574741 7.745234
6.8722563 8.066827
6.6224127 15.180672
10.685385 14.79028
9.847601 13.807644
11.233088 12.084132
7.9840612 10.038246
10.12052 10.328323
11.335262 11.335262
8.932191 9.8828745
10.680259 12.207825
11.36256 11.692709
7.6428595 10.231503
12.420116 12.420116
9.465078 11.136389
13.003281 14.111603
9.124323 14.354223
12.917548 13.263089
10.229958 10.229958
11.024193 11.378474
8.508591 11.00717
8.8715515 9.98302
9.032919 18.165508
11.3465805 14.005871
1.6634805 12.529781
2.1451225 13.803011
3.5987897 13.024347
10.999809 13.145205
2.5411954 11.922661


5.759408 7.9229975
7.6367326 8.098164
7.14879 7.14879
9.223058 9.853012
7.184747 8.215717
8.279452 8.34589
11.540165 13.536065
9.32662 9.740079
2.3721309 11.6369
6.9522204 11.10928
8.9792385 11.484659
8.982863 10.085493
11.208178 11.208178
11.105663 12.248779
10.341805 10.341805
11.898858 15.152819
3.6374674 17.249477
7.842474 15.632244
6.63122 15.959875
-2.640955 16.926004
7.8767433 17.249426
6.689848 15.922642
8.404281 17.013653
8.569313 16.67193
3.740457 16.569439
7.4059544 11.882225
8.7014065 11.399677
8.123912 9.1514635
8.060131 9.877815
8.060199 8.750818
8.428539 9.422111
8.407121 12.05376
12.090097 12.090097
10.610954 13.579565
10.169038 13.551179
7.9280853 13.130985
12.722907 13.491043
8.108944 11.474943
9.7378235 11.434393
9.178339 12.163086
8.535938 11.679036
7.8986807 9.045332
11.114362 12.967664
11.890136 15.160311
11.7063 14.886769
10.71632 12.628244
11.226776 14.249207
12.455587 14.670211
12.090057 12.251509
11.481221 13.518354
11.95744 15.085009
10.160431 16.239634
12.13

In [58]:
values = pd.Series(values)

# TODO - this is a hack to get rid of trajectories that resulted
# in extremely high values, as a result of rho being very large. 
# Think of a better way of doing this - perhaps by restricting
# the actions that the agent can take at any timestep?
# values = (values[(values >= -15) & (values <= 15)])

In [59]:
np.mean(values)

11.829514552151904

In [24]:
# Evaluate the average return under the physician policy on the test set.
phys_vals = []
gamma = 0.99
for uid in unique_ids:
    traj = df_test.loc[df_test['icustayid'] == uid]
    ret = 0
    reversed_traj = traj.iloc[::-1]
    for row in reversed_traj.index:
        ret = reversed_traj.loc[row,'reward'] + gamma*ret
    phys_vals.append(ret)
np.mean(phys_vals)

9.96289026695225